In [ ]:
import io
import socket
import struct
from PIL import Image
import cv2
import numpy as np
from tensorflow.keras.models import model_from_json

WEIGHTS_FILE = "model-0.87.h5" # Loading weights
EPSILON = 0.02
IMAGES = []

# Start a socket listening for connections on 0.0.0.0:8000 (0.0.0.0 means
# all interfaces)
server_socket = socket.socket()
server_socket.bind(('192.168.0.109', 8000))
server_socket.listen(0)

# Accept a single connection and make a file-like object out of it
connection = server_socket.accept()[0].makefile('rb')
try:
    while True:
        # Read the length of the image as a 32-bit unsigned int. If the
        # length is zero, quit the loop
        image_len = struct.unpack('<L', connection.read(struct.calcsize('<L')))[0]
        if not image_len:
            break
        # Construct a stream to hold the image data and read the image
        # data from the connection
        image_stream = io.BytesIO()
        image_stream.write(connection.read(image_len))
        # Rewind the stream, open it as an image with PIL and do some
        # processing on it
        image_stream.seek(0)

        file_bytes = np.asarray(bytearray(image_stream.read()), dtype=np.uint8)
        image = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
        print('Image resived')
        # cv2.imshow("img", image)
        # cv2.waitKey(0)

        IMAGES.append(image)
        
finally:
    connection.close()
    server_socket.close()
    
    json_file = open("cnn.json", "r")
    loaded_model_json = json_file.read()
    json_file.close()
    # Создаем модель на основе загруженных данных
    model = model_from_json(loaded_model_json)
    # model = create_model()
    model.load_weights(WEIGHTS_FILE)

    for filename in IMAGES:
        unscaled = cv2.imread(filename)
        copy_unscaled = np.copy(unscaled)
        image = cv2.resize(unscaled, (IMAGE_SIZE, IMAGE_SIZE))
        feat_scaled = preprocess_input(np.array(image, dtype=np.float32))

        region = np.squeeze(model.predict(feat_scaled[np.newaxis,:]))

        output = np.zeros(region.shape, dtype=np.uint8)
        output[region > 0.5] = 1
        
        __, contours, _ = cv2.findContours(output, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            approx = cv2.approxPolyDP(cnt, EPSILON * cv2.arcLength(cnt, True), True)
            x, y, w, h = cv2.boundingRect(approx)
            
            # Coordinates of objects
            x0 = np.rint(x * unscaled.shape[1] / output.shape[1]).astype(int)
            x1 = np.rint((x + w) * unscaled.shape[1] / output.shape[1]).astype(int)
            y0 = np.rint(y * unscaled.shape[0] / output.shape[0]).astype(int)
            y1 = np.rint((y + h) * unscaled.shape[0] / output.shape[0]).astype(int)
            # Selection by area
            if (x1-x0)*(y1-y0) > 20000:
                cv2.rectangle(unscaled, (x0, y0), (x1, y1), (0, 128, 0), 2)
                cv2.line(unscaled, (x0+(x1-x0)//2, y0), (x0+(x1-x0)//2, y1), (255, 0, 0))
                cv2.line(unscaled, (x0, y0+(y1-y0)//2), (x1, y0+(y1-y0)//2), (255, 0, 0))
                
                cx, cy = x0+(x1-x0)//2, y0+(y1-y0)//2 # Coordinates of center
                cv2.putText(unscaled, 
                            f"x: {cx}, y: {cy}", 
                            (cx + 10, cy - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 
                            0.5, (128, 0, 255), 
                            lineType=cv2.LINE_AA)
                
                # Calculating center shift
                h, w = unscaled.shape[:2]
                shift = w//2 - cx
                cv2.putText(unscaled, 
                            f"Move on: {shift}", 
                            (4, 20), 
                            cv2.FONT_HERSHEY_SIMPLEX, 
                            0.8, (100, 0, 255), 
                            lineType=cv2.LINE_AA)
                
                # Crop image for classification
                crop = copy_unscaled[y0:y1, x0:x1]
                # cv2.imshow("crop", crop)
                
                '''classification'''

        cv2.imshow("image", unscaled)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
